In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel, AdamW
import numpy as np
import pandas as pd
from tqdm import tqdm
import xml.etree.ElementTree as ET
import nltk
from nltk.corpus import wordnet as wn
from collections import namedtuple

In [2]:
import numpy as np

def generate_balls(num_layers, current_idx, current_center, current_radius, balls, dim):
    if num_layers == 0:
        return

    # Define the distance between the centers of two sibling balls
    separation = 2 * current_radius / 3

    # Choose a random direction for the first child ball's center
    direction = np.random.randn(dim)
    direction = direction / np.linalg.norm(direction)  # Normalize the direction vector

    # Calculate the centers for the two child balls
    center1 = current_center + direction * separation
    center2 = current_center - direction * separation

    # Calculate the radius for the child balls
    child_radius = current_radius / 3

    # Assign the child balls to the dictionary with updated indices
    left_idx = 2 * current_idx + 1
    right_idx = 2 * current_idx + 2
    balls[left_idx] = {'center': center1, 'radius': child_radius}
    balls[right_idx] = {'center': center2, 'radius': child_radius}

    # Recursive call for each child
    generate_balls(num_layers - 1, left_idx, center1, child_radius, balls, dim)
    generate_balls(num_layers - 1, right_idx, center2, child_radius, balls, dim)

# Root ball parameters
dim = 3
layer = 4
initial_radius = 10
initial_center = np.zeros(dim)

# Initialize the dictionary to hold all the balls
balls = {0: {'center': initial_center, 'radius': initial_radius}}

# Generate the balls recursively
generate_balls(layer, 0, initial_center, initial_radius, balls, dim)

# Check how many balls we have generated
ball_num = len(balls)
print(ball_num)

31


In [3]:
# Settings for loading training data
semcor_training_xml_path = 'WSD_Evaluation_Framework/Training_Corpora/Semcor/semcor.data.xml'
semcor_training_gk_path = 'WSD_Evaluation_Framework/Training_Corpora/Semcor/semcor.gold.key.txt'

In [4]:
def load_xml_data(xml_file_path=''):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    data = []
    for text in root.findall('text'):
        for sentence in text.findall('sentence'):
            sentence_id = sentence.get('id')
            sentence_text = ' '.join([element.text for element in sentence])
            for instance in sentence.findall('instance'):
                instance_id = instance.get('id')
                lemma = instance.get('lemma')
                pos = instance.get('pos')
                word = instance.text
                data.append([sentence_id, instance_id, lemma, pos, word, sentence_text])
    columns = ['sentence_id', 'instance_id', 'lemma', 'pos', 'word', 'sentence_text']
    xml_data = pd.DataFrame(data, columns=columns)
    return xml_data


# Load xml training data from semcor
semcor_training_xml = load_xml_data(semcor_training_xml_path)
display(semcor_training_xml.head())

,sentence_id,instance_id,lemma,pos,word,sentence_text
0,d000.s000,d000.s000.t000,long,ADJ,long,How long has it been since you reviewed the ob...
1,d000.s000,d000.s000.t001,be,VERB,been,How long has it been since you reviewed the ob...
2,d000.s000,d000.s000.t002,review,VERB,reviewed,How long has it been since you reviewed the ob...
3,d000.s000,d000.s000.t003,objective,NOUN,objectives,How long has it been since you reviewed the ob...
4,d000.s000,d000.s000.t004,benefit,NOUN,benefit,How long has it been since you reviewed the ob...


In [5]:
def load_gold_keys(gold_key_file_path=''):
    gold_key_data = []
    with open(gold_key_file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            instance_id = parts[0]
            sense_id = parts[1]
            gold_key_data.append([instance_id, sense_id])

    # Create a DataFrame
    gold_key_columns = ['instance_id', 'sense_id']
    gold_key_df = pd.DataFrame(gold_key_data, columns=gold_key_columns)
    return gold_key_df

# Load gold key training data from semcor
semcor_training_gk = load_gold_keys(semcor_training_gk_path)
display(semcor_training_gk.head())

,instance_id,sense_id
0,d000.s000.t000,long%3:00:02::
1,d000.s000.t001,be%2:42:03::
2,d000.s000.t002,review%2:31:00::
3,d000.s000.t003,objective%1:09:00::
4,d000.s000.t004,benefit%1:21:00::


In [6]:
# Merge data from two file
semcor_training_merged = pd.merge(semcor_training_xml, semcor_training_gk, on='instance_id', how='inner')
display(semcor_training_merged.head())

,sentence_id,instance_id,lemma,pos,word,sentence_text,sense_id
0,d000.s000,d000.s000.t000,long,ADJ,long,How long has it been since you reviewed the ob...,long%3:00:02::
1,d000.s000,d000.s000.t001,be,VERB,been,How long has it been since you reviewed the ob...,be%2:42:03::
2,d000.s000,d000.s000.t002,review,VERB,reviewed,How long has it been since you reviewed the ob...,review%2:31:00::
3,d000.s000,d000.s000.t003,objective,NOUN,objectives,How long has it been since you reviewed the ob...,objective%1:09:00::
4,d000.s000,d000.s000.t004,benefit,NOUN,benefit,How long has it been since you reviewed the ob...,benefit%1:21:00::


In [7]:
sense_id_cache = {}
def format_sense_id(sense_id):
    if sense_id not in sense_id_cache:
        sense_id_cache[sense_id] = wn.lemma_from_key(sense_id).synset().name()
    return sense_id_cache[sense_id]
    
semcor_training_merged['formatted_sense_id'] = semcor_training_merged['sense_id'].apply(format_sense_id)

# We keep those columns for now
keys_to_keep = ['lemma', 'word', 'sentence_text', 'formatted_sense_id']
semcor_training_merged = semcor_training_merged[keys_to_keep]

display(semcor_training_merged.head())

,lemma,word,sentence_text,formatted_sense_id
0,long,long,How long has it been since you reviewed the ob...,long.a.01
1,be,been,How long has it been since you reviewed the ob...,be.v.01
2,review,reviewed,How long has it been since you reviewed the ob...,review.v.01
3,objective,objectives,How long has it been since you reviewed the ob...,aim.n.02
4,benefit,benefit,How long has it been since you reviewed the ob...,benefit.n.01


In [8]:
unique_sense_ids = semcor_training_merged['formatted_sense_id'].unique()
print(len(unique_sense_ids))
sampled_ids = pd.Series(unique_sense_ids).sample(n=ball_num, random_state=1)
print(len(sampled_ids))
semcor_training = semcor_training_merged[semcor_training_merged['formatted_sense_id'].isin(sampled_ids)]
print(len(semcor_training))

25916
31
229


In [9]:
# Setting for the model choice
models = {
    "BERT-Base": "bert-base-uncased",
    "BERT-Large": "bert-large-uncased",
    "BERT-Medium": "google/bert_uncased_L-8_H-512_A-8",
    "BERT-Small": "google/bert_uncased_L-4_H-256_A-4",
    "BERT-Mini": "google/bert_uncased_L-4_H-128_A-2",
    "BERT-Tiny": "google/bert_uncased_L-2_H-128_A-2"
}

# With our nball dimention 162, we choose bert small with 256 dimentions
model_name = models["BERT-Small"]

In [10]:
# Get sense index
sense_labels = list(sampled_ids)
sense_index_num = list(balls.keys())
sense_index = {sense: idx for idx, sense in zip(sense_index_num, sense_labels)}
semcor_training.loc[:,'sense_idx'] = semcor_training['formatted_sense_id'].map(sense_index)

original_dim = len(balls[sense_index_num[0]]['center'])
target_dim = 3  # Dimension of BERT-Small
padding_size = target_dim - original_dim  # Adjust padding size based on the new dimension

# Process each embedding
padded_embeddings = []
nball_radius = []
scalling_factor = 1
for index in sense_index_num:
    # Pad the trimmed embedding to match the target dimension
    padded_embedding = np.pad(balls[index]['center'], (0, padding_size), 'constant', constant_values=0)
    # Add to the list of padded embeddings
    padded_embeddings.append(scalling_factor*padded_embedding)
    nball_radius.append(scalling_factor*balls[index]['radius'])

sense_embeddings = torch.tensor(np.array(padded_embeddings), dtype=torch.float64)
nball_radius = torch.tensor(np.array(nball_radius), dtype=torch.float64)

print(f'Total data of nball embeddings:{len(sense_embeddings)}')
print(f'The length after padding: {len(sense_embeddings[0])}')

Total data of nball embeddings:31
The length after padding: 3


D:\Anaconda\envs\annotated-transformer\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [11]:
# Tokenize the sentence
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Could be problem here, as we always fid
def find_word_index(sentence_ids, word):
    word_tokens = tokenizer.tokenize(word)
    word_ids = tokenizer.convert_tokens_to_ids(word_tokens)
    for i in range(len(sentence_ids) - len(word_tokens) + 1):
        if sentence_ids[i:i+len(word_tokens)].tolist() == word_ids:
            return i
    return -1

def tokenize_data(df):
    # Tokenize all sentences
    print("Tokenizing sentences...")
    tokenized_data = tokenizer(list(df['sentence_text']), padding=True, truncation=True, return_tensors="pt", max_length=512)
    input_ids = tokenized_data['input_ids']
    attention_masks = tokenized_data['attention_mask']

    # Progress bar for calculating word indices
    print("Calculating word indices...")
    # pbar = tqdm(total=df.shape[0], desc="Calculating word indices")
    word_indices = []
    for sentence_ids, word in zip(input_ids, df['word']):
        word_indices.append(find_word_index(sentence_ids, word))
        # pbar.update(1)  # Update progress for each word index found

    # print(f"Length input_ids:{len(input_ids)}\n Length attention_mask:{len(attention_masks)}\n Length \
    # word_index:{len(word_indices)}\n Length dataframe:{len(df)}")
    df.loc[:, 'input_ids'] = input_ids.tolist()
    df.loc[:, 'attention_mask'] = attention_masks.tolist()
    df.loc[:, 'word_index'] = word_indices

    print('Tokenizing finished!')
    # pbar.close()  # Close the progress bar after completion


tokenize_data(semcor_training)
display(semcor_training.head())

Tokenizing sentences...
Calculating word indices...
Tokenizing finished!


,lemma,word,sentence_text,formatted_sense_id,sense_idx,input_ids,attention_mask,word_index
985,hot,hot,Latest models serve hot meals at reasonable pr...,hot.a.01,11,"[101, 6745, 4275, 3710, 2980, 12278, 2012, 960...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4
2702,partly,partly,The trouble was at least partly Juet 's doing .,partially.r.01,18,"[101, 1996, 4390, 2001, 2012, 2560, 6576, 1841...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",6
3176,experience,experience,"On the one hand , the major European nations h...",experience.n.03,20,"[101, 2006, 1996, 2028, 2192, 1010, 1996, 2350...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",66
6558,enclosure,enclosure,Sometimes these servants wrote or dictated for...,enclosure.n.02,9,"[101, 2823, 2122, 8858, 2626, 2030, 23826, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",8
8084,hot,hot,"Eyes like hot honey , eyes that sizzled .",hot.a.01,11,"[101, 2159, 2066, 2980, 6861, 1010, 2159, 2008...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",3


In [12]:
# Setting, small batch size here so that do not out of cuda memory
batch_size = 32

# Create tensor dataset to speed learning
# Convert lists of lists into tensors
all_input_ids = torch.stack([torch.tensor(ids, dtype=torch.long) for ids in semcor_training['input_ids']])
all_attention_masks = torch.stack([torch.tensor(mask, dtype=torch.long) for mask in semcor_training['attention_mask']])
all_word_indices = torch.tensor(semcor_training['word_index'].tolist(), dtype=torch.long)
all_senses = torch.tensor(semcor_training['sense_idx'].tolist(), dtype=torch.long)

# Create a TensorDataset
dataset = TensorDataset(all_input_ids, all_attention_masks, all_word_indices, all_senses)

# Use DataLoader to handle batching
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [13]:
print(nball_radius.median())
print(len(nball_radius))

tensor(0.1235, dtype=torch.float64)
31


In [14]:
# Training the model
# Loss
from torch.optim.lr_scheduler import StepLR
import math

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# top_values, top_indices = torch.topk(nball_radius, 10)
# top_indices = top_indices.to(device)

def ball_inclusion_loss(word_embeddings, centers, radius, labels, epsilon=1e-7):
    distances = torch.norm(word_embeddings - centers, dim=1)
    # # Compute a binary mask for embeddings within the ball radius
    # # print(f"difference:{radius-distances}")
    within_ball = (distances <= radius).float()
    within_ball = torch.clamp(within_ball, epsilon, 1 - epsilon)
    loss = F.binary_cross_entropy(within_ball, labels.float(), reduction='mean')
    return loss, distances

def ball_inclution_loss_soft_margin(word_embeddings, centers, radius):
    distances = torch.norm(word_embeddings - centers, dim=1)
    soft_margin = torch.clamp(distances - radius, min=0)
    loss = torch.mean(soft_margin ** 2)
    return loss, distances
 

class CustomBertModel(nn.Module):
    def __init__(self, bert_model_name):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.projection = nn.Linear(self.bert.config.hidden_size, 3)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        projected_hidden_states = self.projection(hidden_states)
        return projected_hidden_states


# Model setting
# model = BertModel.from_pretrained(model_name).to(device)
model = CustomBertModel("google/bert_uncased_L-2_H-128_A-2").to(device)
model.train() 
sense_embeddings = sense_embeddings.to(device)  # Move sense embeddings to GPU
# loss_fn = ball_inclusion_loss
# loss_fn = nn.MSELoss()
loss_fn = ball_inclution_loss_soft_margin
optimizer = optim.Adam([
    {'params': model.parameters()}
    # {'params': sense_embeddings},
    # {'params': nball_radius}
], lr=2e-3)

scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


# Training loop
num_epochs = 50
last_loss = 0
for epoch in range(num_epochs):
    total_loss = 0
    total_dis = 0
    total_radius = 0
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True, position=0)
    for batch in progress_bar:
        # Send batch data to the device (GPU)
        batch_input_ids, batch_attention_masks, batch_word_indices, batch_sense_indices = [b.to(device) for b in batch]

        optimizer.zero_grad()
        
        # Forward pass
        # outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
        # hidden_states = outputs.last_hidden_state
        hidden_states = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
        # Retrieve embeddings for specific word indices
        word_embeddings = torch.stack([hidden_states[i, idx, :] for i, idx in enumerate(batch_word_indices)])
        
        # Retrieve the corresponding sense embeddings
        center = sense_embeddings[batch_sense_indices].to(dtype=torch.float32).to(device)
        radius = nball_radius[batch_sense_indices].to(device)

        # Labels tensor indicating that embeddings should be inside the ball
        labels = torch.ones(word_embeddings.size(0), device=device, requires_grad=True)
        # print("word_embeddings requires grad:", word_embeddings.requires_grad)
        # print("centers requires grad:", center.requires_grad)

        # # You can also check for the outputs from the model
        # print("Model labels requires grad:", labels.requires_grad)

        # Calculate loss
        # loss, dis = loss_fn(word_embeddings, center, radius, labels, epsilon=1e-2)
        loss, dis = loss_fn(word_embeddings, center, radius)
        # loss = loss_fn(word_embeddings, center)
        # dis = nn.MSELoss()(word_embeddings, center)
        # distances = torch.norm(word_embeddings - center, dim=1)
        # Backward pass
        loss.backward()

        # for name, param in model.named_parameters():
        #     if param.requires_grad and param.grad is not None:
        #         print(f"Gradient of {name}: {param.grad.abs().mean().item()}")
        #     elif param.requires_grad:
        #         print(f"No gradient for {name}")

        
        optimizer.step()
        
        total_loss += loss.item()
        total_dis += dis.sum().item()
        total_radius += radius.sum().item()
        progress_bar.set_postfix(loss=loss.item())

    
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}, improve:{(last_loss - total_loss) / len(dataloader)}')
    print(f'distance:{total_dis / len(dataloader)}, radius:{total_radius / len(dataloader)}, differences:{(total_dis-total_radius) / len(dataloader)}')
    last_loss = total_loss

Using device: cuda


Epoch 1/50: 100%|█████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.31it/s, loss=7.24]


Epoch 1, Loss: 27.774158314282776, improve:-27.774158314282776
distance:160.05121088027954, radius:18.194444444444446, differences:141.85676643583508


Epoch 2/50: 100%|█████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 44.05it/s, loss=3.72]


Epoch 2, Loss: 7.437861846845153, improve:20.336296467437624
distance:73.268807888031, radius:18.194444444444446, differences:55.07436344358656


...

Epoch 48, Loss: 0.01812447097667686, improve:0.0038517270696944994
distance:11.479848027229309, radius:18.194444444444443, differences:-6.714596417215134


Epoch 49/50: 100%|████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 42.76it/s, loss=0.02]


Epoch 49, Loss: 0.020855560831372577, improve:-0.002731089854695716
distance:12.022155493497849, radius:18.194444444444446, differences:-6.172288950946598


Epoch 50/50: 100%|█████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 36.88it/s, loss=0.00469]

Epoch 50, Loss: 0.018154324748705347, improve:0.00270123608266723
distance:11.452143862843513, radius:18.194444444444443, differences:-6.742300581600929


In [15]:
# Assuming that 'dataloader' for evaluation is available as eval_dataloader

# Switch model to evaluation mode
eval_dataloader = dataloader
model.eval()

# Store evaluation results
eval_loss = 0
accuracy = 0  # Placeholder for accuracy calculation

not_correct = []
# Disable gradient computation for evaluation to save memory and computations
with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluation", leave=True, position=0):
        # Send batch data to the device (GPU)
        batch_input_ids, batch_attention_masks, batch_word_indices, batch_sense_indices = [b.to(device) for b in batch]

        # Forward pass
        hidden_states = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
        # hidden_states = outputs.last_hidden_state

        # Retrieve embeddings for specific word indices
        word_embeddings = torch.stack([hidden_states[i, idx, :] for i, idx in enumerate(batch_word_indices)])

        # Retrieve the corresponding sense embeddings and radius
        center = sense_embeddings[batch_sense_indices].to(dtype=torch.float32).to(device)
        radius = nball_radius[batch_sense_indices].to(device)

        # Labels tensor (usually true for all points in eval since you are evaluating the inclusion)
        labels = torch.ones(word_embeddings.size(0), device=device)

        # Calculate loss
        loss, dis = ball_inclusion_loss(word_embeddings, center, radius, labels)
        eval_loss += loss.item()

        # Optionally calculate accuracy or other metrics
        # For instance, if accuracy is based on correct inclusions
        predictions = (torch.norm(word_embeddings - center, dim=1) <= radius).float()
        accuracy += (predictions == labels).float().mean().item()
        incorrect_indices = batch_sense_indices[predictions != labels]
        not_correct.extend(incorrect_indices.cpu().tolist())

# Compute average loss and accuracy
eval_loss /= len(eval_dataloader)
accuracy /= len(eval_dataloader)

print(f"Average Evaluation Loss: {eval_loss}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 125.66it/s]


Average Evaluation Loss: 9.305681765079498
Accuracy: 42.27%


In [16]:
uni_not_correct = set(not_correct)
nball_radius[list(uni_not_correct)]

tensor([0.3704, 0.3704, 0.1235, 0.1235, 0.1235, 0.1235, 0.3704, 0.3704, 0.1235,
        0.1235, 0.1235, 0.1235, 0.3704, 0.3704, 0.1235, 0.1235, 0.1235, 0.1235,
        0.1235, 0.1235, 0.1235, 0.1235], dtype=torch.float64)

In [17]:
nball_radius

tensor([10.0000,  3.3333,  3.3333,  1.1111,  1.1111,  0.3704,  0.3704,  0.1235,
         0.1235,  0.1235,  0.1235,  0.3704,  0.3704,  0.1235,  0.1235,  0.1235,
         0.1235,  1.1111,  1.1111,  0.3704,  0.3704,  0.1235,  0.1235,  0.1235,
         0.1235,  0.3704,  0.3704,  0.1235,  0.1235,  0.1235,  0.1235],
       dtype=torch.float64)